## 🔴 Ejercicio Avanzado
Implementa un proceso completo de sliding window para `fact_ventas`:
1. Crear tabla particionada por mes (últimos 36 meses)
2. Job mensual que:
   - Archiva partición más antigua (SWITCH a tabla de archivo)
   - Crea nueva partición para el mes entrante (SPLIT)
3. Script SQL con manejo de errores y logging

## Errores Comunes

❌ **Particionar tablas pequeñas**: overhead sin beneficio si tabla < 100M filas
❌ **Columna de partición mal elegida**: debe ser la que más filtremos (usualmente fecha)
❌ **No alinear índices**: índices deben estar en el mismo partition scheme
❌ **Demasiadas particiones**: >1000 particiones degrada metadata, elige granularidad apropiada
❌ **No planificar crecimiento**: olvidan agregar nuevas particiones futuras

---

## Cuándo Particionar

| Escenario | ¿Particionar? |
|-----------|--------------|
| Tabla 10M filas, queries usan fecha | ❌ No, índice basta |
| Tabla 500M filas, archivar mensual | ✅ Sí, por mes |
| Tabla 1B filas, solo queries últimos 90 días | ✅ Sí, por día |
| Tabla 5M filas, OLTP transaccional | ❌ No, overhead excesivo |

**Siguiente:** `05_transacciones_bloqueos.ipynb` → ACID, isolation levels, deadlocks

## Consultar metadata de particiones

```sql
-- Ver en qué partición cae una fecha específica
SELECT $PARTITION.pf_ventas_anio('2024-06-15') AS numero_particion;

-- Ver filas por partición
SELECT $PARTITION.pf_ventas_anio(fecha) AS particion, COUNT(*) AS filas
FROM fact_ventas_particionada
GROUP BY $PARTITION.pf_ventas_anio(fecha)
ORDER BY particion;
```

## 🟢 Ejercicio Básico
Explica conceptualmente: si tienes particiones por mes y ejecutas `WHERE fecha >= '2024-01-01' AND fecha < '2024-02-01'`, ¿cuántas particiones escaneará SQL Server? ¿Por qué esto es más rápido que una tabla sin particionar?

## 🟠 Ejercicio Intermedio
Diseña una estrategia de particionamiento para `fact_suscripciones` con 200M filas, particionada por trimestre. Define:
1. Partition function (límites)
2. Criterio de archivado (eliminar trimestres > 5 años)
3. Proceso de mantenimiento mensual (agregar nueva partición)

In [ ]:
-- Ejemplo conceptual de particionamiento por año (no ejecutar en dataset pequeño)

-- Paso 1: Crear función de partición (define límites)
-- CREATE PARTITION FUNCTION pf_ventas_anio (DATE)
-- AS RANGE RIGHT FOR VALUES ('2022-01-01', '2023-01-01', '2024-01-01', '2025-01-01');
-- Esto crea 5 particiones:
-- P1: fecha < 2022-01-01
-- P2: 2022-01-01 <= fecha < 2023-01-01
-- P3: 2023-01-01 <= fecha < 2024-01-01
-- P4: 2024-01-01 <= fecha < 2025-01-01
-- P5: fecha >= 2025-01-01

-- Paso 2: Crear esquema de partición (mapea a filegroups)
-- CREATE PARTITION SCHEME ps_ventas_anio
-- AS PARTITION pf_ventas_anio
-- ALL TO ([PRIMARY]); -- En producción usarías filegroups separados

-- Paso 3: Crear tabla particionada
-- CREATE TABLE fact_ventas_particionada (
--     venta_id INT IDENTITY(1,1),
--     fecha DATE NOT NULL,
--     cliente_id INT,
--     producto_id INT,
--     cantidad INT,
--     descuento_pct DECIMAL(5,2)
-- ) ON ps_ventas_anio(fecha); -- La columna fecha determina la partición

/*
💡 Beneficios en acción:
1. Query con WHERE fecha >= '2024-01-01' → solo escanea particiones 4 y 5
2. Archivar 2022: SWITCH partition 2 a tabla de archivo (operación metadata, instantánea)
3. Rebuild índices de 2024: ALTER INDEX ... REBUILD PARTITION = 4 (no bloquea otras particiones)
*/

# 3.3 Particiones Básico - Dividir Tablas Grandes

## 🎯 ¿Para qué?
Particionar una tabla significa dividirla físicamente en fragmentos (particiones) según un criterio (ej. por mes). Esto permite:
- **Archivar datos históricos** eficientemente (eliminar partición completa en segundos vs DELETE lento)
- **Queries más rápidas** si filtran por la columna de partición
- **Mantenimiento paralelo** (rebuild índices por partición)

## 📚 ¿Por qué particionar?

Imagina `fact_ventas` con 500M de filas de 5 años. Sin particiones:
- `DELETE FROM fact_ventas WHERE fecha < '2020-01-01'` → tarda horas, genera log masivo
- Rebuild de índices → bloquea toda la tabla

Con particiones por año:
- `ALTER TABLE fact_ventas SWITCH PARTITION 1 TO fact_ventas_archive` → segundos
- Rebuild partition 5 (año actual) → no bloquea años anteriores

## 🔧 ¿Cómo funciona?

1. **Partition Function**: define los rangos (ej. límites por mes)
2. **Partition Scheme**: mapea rangos a filegroups (opcional)
3. **Tabla particionada**: se crea/altera sobre el scheme

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.